In [35]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from model import Net

In [36]:
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [37]:
test_loader = DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1)

In [38]:
model = Net()
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # I don't have gpu :(

loss_fn = nn.CrossEntropyLoss()

def test():
    model.eval()

    test_loss = 0
    correct = 0

    true_positives = {i: 0 for i in range(10)}  # 10 classes for digits 0-9
    total_predicted_positives = {i: 0 for i in range(10)}


    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred =  output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

            for i in range (10):
                # Calculate true positives for class i
                # true_positives[i] += ((pred == i) & (target == i)).sum().item()
                # true_positives[i] += ((pred == i) & (target == i))
                # Calculate total predicted positives for class i
                # total_predicted_positives[i] += (pred == i).sum().item()
                # total_predicted_positives[i] += (pred == i & 1)

                for j in range (len(pred)):
                    if pred[j] == i:
                        total_predicted_positives[i] += 1
                        if target[j] == i:
                            true_positives[i] += 1

    test_loss /= len(test_loader.dataset)

    print(f'Average loss: {test_loss:.4f}')
    print(f'Accuracy: {correct} / {len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):0.0f}%)')
    
    # Calculating the precision of each class and printing it
    precision = {}
    for i in range(10):
        if total_predicted_positives[i] == 0:
            precision[i] = 0.0
        else:
            precision[i] = true_positives[i] / total_predicted_positives[i]

        print(f'Precision for class {i}: {precision[i]}')

In [40]:
test()

/Users/kc/digitRecognition/model.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Average loss: 0.0149
Accuracy: 9760 / 10000 (98%)
Precision for class 0: 0.971028971028971
Precision for class 1: 0.9929203539823008
Precision for class 2: 0.9691119691119691
Precision for class 3: 0.9781094527363184
Precision for class 4: 0.9777777777777777
Precision for class 5: 0.9807909604519774
Precision for class 6: 0.9822175732217573
Precision for class 7: 0.9587727708533078
Precision for class 8: 0.9730569948186528
Precision for class 9: 0.9757330637007078
